In [15]:
#load csv
import csv
import pandas as pd
import numpy as np
import os

#set path
pwd = os.getcwd()
file_name = 'yellow_tripdata_2016-03.csv'
csvpath = os.path.join(pwd, 'Kaggle/'+file_name)

#open csv and convert to dataframe
df = pd.read_csv(csvpath)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [16]:
# data cleaning
# extract only the columns we need
df = df[['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude','dropoff_latitude']]

In [17]:
# rank ascending by pickup time
df = df.sort_values(by='tpep_pickup_datetime')

# remove rows with missing values
df = df.dropna()

# slice the data into different days based on the pickup time
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['day'] = df['tpep_pickup_datetime'].dt.day
df['month'] = df['tpep_pickup_datetime'].dt.month
df['year'] = df['tpep_pickup_datetime'].dt.year

# seprate the data into different dataframes based on the day
for day in range(1,32,1):
    DF_NAME = 'df_day_'+str(day)
    globals()[DF_NAME] = df[df['day'] == day]
    

In [18]:
df_days = [df_day_1, df_day_2, df_day_3, df_day_4, df_day_5, df_day_6, df_day_7, df_day_8, df_day_9, df_day_10, df_day_11, df_day_12, df_day_13, df_day_14, df_day_15, df_day_16, df_day_17, df_day_18, df_day_19, df_day_20, df_day_21, df_day_22, df_day_23, df_day_24, df_day_25, df_day_26, df_day_27, df_day_28, df_day_29, df_day_30, df_day_31]

In [19]:
# Mapping the lat/long to the node ID
import scipy.io as scipy
import os.path as osp
MAP_NAME = 'NYC_Manhattan_Map.mat'
map = scipy.loadmat(os.path.join(pwd, MAP_NAME))
map_Nodes = pd.DataFrame(map['Nodes']) # Node ID, latitude, longtitude

In [20]:
map_Nodes.rename(columns={0:'NodeID',1:'Latitude',2:'Longitude'},inplace=True)
map_Nodes['NodeID'] = map_Nodes['NodeID'].astype(int)
map_Nodes.loc[map_Nodes['NodeID'] == 3019, 'Longitude'] = -73.950005

map_Nodes.head()

,NodeID,Latitude,Longitude
0,1,40.706991,-74.017946
1,2,40.706175,-74.017930
2,3,40.707914,-74.017808
3,4,40.706840,-74.017575
4,5,40.707624,-74.017503


In [21]:
# Create a dict to map lat/long to node ID
map_dict = {}
for i in range(len(map_Nodes)):
    map_dict[(map_Nodes['Latitude'][i],map_Nodes['Longitude'][i])] = map_Nodes['NodeID'][i]

def get_NodeID(lat, long):
    if (lat,long) in map_dict:
        return map_dict[(lat,long)]
    else:
        return None


In [22]:
from scipy.spatial import cKDTree
# if no exact match, find the closest node
def find_nearest_nodes(df1, df2):
    
    # 检查输入数据框是否包含必要的列
    required_cols_df1 = {'Latitude', 'Longitude', 'NodeID'}
    required_cols_df2 = {'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'}
    
    if not all(col in df1.columns for col in required_cols_df1):
        raise ValueError(f"df1必须包含以下列: {required_cols_df1}")
    if not all(col in df2.columns for col in required_cols_df2):
        raise ValueError(f"df2必须包含以下列: {required_cols_df2}")
    
    # 创建KD树用于快速最近邻搜索
    tree = cKDTree(df1[['Latitude', 'Longitude']])
    
    # 对每个df2中的点找到最近的节点
    _, PU_indices = tree.query(df2[['pickup_latitude', 'pickup_longitude']], k=1)
    _, DO_indices = tree.query(df2[['dropoff_latitude', 'dropoff_longitude']], k=1)
    
    # 创建结果DataFrame
    result = df2.copy()
    
    # 添加最近节点的信息
    result['PU_nodeID'] = df1.iloc[PU_indices]['NodeID'].values
    result['DO_nodeID'] = df1.iloc[DO_indices]['NodeID'].values

    return result


In [23]:
def convert_date_to_sec(df):
    date = df['tpep_pickup_datetime']
    date = pd.to_datetime(date)
    time = date.dt.hour*3600 + date.dt.minute*60 + date.dt.second
    df['ReqTime'] = time
    return df

In [24]:
clean_dfs = []
for df_day in df_days:
    result = find_nearest_nodes(map_Nodes, df_day)
    df_day['Oid'] = result['PU_nodeID']
    df_day['Did'] = result['DO_nodeID']
    df_day = convert_date_to_sec(df_day)
    #rename columns
    df_day.rename(columns={'passenger_count':'Size'}, inplace=True)
    df_day.insert(0, 'ReqID', range(1, 1 + len(df_day)))
    clean_df = df_day[['ReqID', 'Oid', 'Did', 'ReqTime', 'Size']]
    clean_dfs.append(clean_df)


C:\Users\18193\AppData\Local\Temp\ipykernel_33744\1060884915.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_day['Oid'] = result['PU_nodeID']
C:\Users\18193\AppData\Local\Temp\ipykernel_33744\1060884915.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_day['Did'] = result['DO_nodeID']
C:\Users\18193\AppData\Local\Temp\ipykernel_33744\1672631707.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [25]:
# save the data
for i in range(len(clean_dfs)):
    clean_dfs[i].to_csv('Kaggle/clean_data/NYC_full_day_'+str(i+1)+'.csv', index=False)